# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Loan Data once again - this time the strutured data available through the CSV!

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/complaints.csv",
    metadata_columns=[
      "Date received", 
      "Product", 
      "Sub-product", 
      "Issue", 
      "Sub-issue", 
      "Consumer complaint narrative", 
      "Company public response", 
      "Company", 
      "State", 
      "ZIP code", 
      "Tags", 
      "Consumer consent provided?", 
      "Submitted via", 
      "Date sent to company", 
      "Company response to consumer", 
      "Timely response?", 
      "Consumer disputed?", 
      "Complaint ID"
    ]
)

loan_complaint_data = loader.load()

for doc in loan_complaint_data:
    doc.page_content = doc.metadata["Consumer complaint narrative"]

Let's look at an example document to see if everything worked as expected!

In [4]:
loan_complaint_data[0]

Document(metadata={'source': './data/complaints.csv', 'row': 0, 'Date received': '03/27/25', 'Product': 'Student loan', 'Sub-product': 'Federal student loan servicing', 'Issue': 'Dealing with your lender or servicer', 'Sub-issue': 'Trouble with how payments are being handled', 'Consumer complaint narrative': "The federal student loan COVID-19 forbearance program ended in XX/XX/XXXX. However, payments were not re-amortized on my federal student loans currently serviced by Nelnet until very recently. The new payment amount that is effective starting with the XX/XX/XXXX payment will nearly double my payment from {$180.00} per month to {$360.00} per month. I'm fortunate that my current financial position allows me to be able to handle the increased payment amount, but I am sure there are likely many borrowers who are not in the same position. The re-amortization should have occurred once the forbearance ended to reduce the impact to borrowers.", 'Company public response': 'None', 'Company'

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "LoanComplaints".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [5]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    loan_complaint_data,
    embeddings,
    location=":memory:",
    collection_name="LoanComplaints"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [10]:
naive_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided data, the most common issue with loans appears to be problems related to dealing with your lender or servicer, including receiving bad or incorrect information, mismanagement of loans, and trouble with payment handling. Specific sub-issues frequently documented include errors in loan balances, misapplied payments, wrongful denials of payment plans, and issues with loan transfers without proper notification. Many complaints also involve inaccurate reporting of loan status and interest, and difficulties in making payments or understanding loan terms.\n\nIn summary, the most common issue tends to be **problems with loan servicers or lenders providing incorrect or misleading information, leading to confusion, incorrect loan balances, or payment issues**.'

In [11]:
naive_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the information provided, some complaints were not handled in a timely manner. Specifically, there are multiple instances where responses or resolutions were delayed:\n\n- One complaint received on 03/28/25 by MOHELA was marked as "Not timely," indicating it was handled past the expected timeframe.\n- Several other complaints, such as those filed with Maximus Federal Services, Inc., and EdFinancial Services, show responses marked as "Closed with explanation," and in some cases, the complainants indicate they have been waiting for over a year without resolution.\n- Additionally, multiple complaints mention ongoing issues with no resolution even after many months, highlighting delays in handling.\n\nTherefore, yes, some complaints did not get handled in a timely manner.'

In [12]:
naive_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People primarily failed to pay back their loans due to a combination of factors including:\n\n1. Lack of clear communication and insufficient information from lenders or servicers about loan status, payment requirements, and changes in loan ownership, leading to unintentional delinquency.\n2. Difficulties with managing interest accumulation and understanding how payments are applied, often resulting in interest growing faster than payments can reduce the principal.\n3. Limited or no access to flexible repayment options, such as income-driven plans, or failure to be informed about such programs.\n4. Financial hardships, such as stagnant wages, economic recessions, or personal financial crises like foreclosure or bankruptcy, making payments unaffordable.\n5. Administrative errors or mismanagement by loan servicers, including failure to notify borrowers of overdue payments or loan transfer details, which contribute to missed or delayed payments.\n6. Borrowers being unaware of their repay

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [13]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(loan_complaint_data, )

We'll construct the same chain - only changing the retriever.

In [14]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [15]:
bm25_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

"Based on the provided context, the most common issue with loans appears to be problems related to dealing with lenders or servicers, such as disputes over fees, improper application of payments, or receiving inaccurate information about loan balances and terms. A recurring theme is borrowers' frustration with miscommunications, incorrect information, or perceived unfair practices by loan servicers."

In [16]:
bm25_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided context, all the complaints listed were responded to by the companies, and the responses are marked as "Closed with explanation" with the response time marked as "Yes" for timeliness. Therefore, according to this information, there are no complaints indicated as not being handled in a timely manner.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People have failed to pay back their loans for various reasons, including issues with payment plans, miscommunication or lack of communication from the loan servicers, errors or problems with automating payments, and difficulties in obtaining forbearance or deferment. Some specific issues from the complaints include being steered into incorrect types of forbearances, lack of responses after applying for relief, and payment reversals due to alleged errors with the bank or the servicer. Additionally, some borrowers were not properly informed about changes in their account status or the transfer of their loans to different servicers, which led to missed payments and negative impacts on their credit scores. Overall, the failures often stem from administrative errors, poor communication, and mishandling by loan servicers.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">
### Answer:

-
- 

</div>

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [18]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [19]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [20]:
contextual_compression_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided context, the most common issue with loans appears to be problems related to dealing with lenders or servicers, such as receiving bad or incorrect information, errors in loan balances, misapplied payments, wrongful denials of payment plans, and mishandling or mishandling of loan data. Many complaints involve disputes over loan balances, lack of proper communication, incorrect information, unauthorized transfers, and privacy violations.'

In [21]:
contextual_compression_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the information provided, one complaint explicitly mentions that it has been nearly 18 months with no resolution, indicating that it was not handled in a timely manner. Additionally, the complaint about the main issue not being addressed and ongoing difficulties with delayed responses suggests that some complaints did not get resolved promptly. \n\nHowever, for the other complaints, the responses from the companies indicate they responded within a timeframe labeled as "Yes" for timely response, although the actual resolution status varies.\n\nTherefore, yes, at least one complaint did not get handled in a timely manner, and there are indications that some issues remained unresolved over extended periods.'

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for several reasons, including:\n\n1. Lack of Awareness and Information: Borrowers were often not informed about the requirement to repay loans or how the loan process works, leading to confusion and unawareness of their repayment obligations.\n\n2. Poor Communication from Servicers: Borrowers reported that lenders or loan servicers failed to notify them when payments were due, did not provide clear information about payment plans, and did not confirm transfers or account details properly.\n\n3. Financial Hardships and Unmanageable Interest: Many borrowers found themselves unable to afford payments because of accumulating interest and the limited options provided, such as forbearance or deferment, which often resulted in interest continuing to grow and increasing overall debt.\n\n4. Economic and Employment Factors: Unexpected economic challenges, stagnant wages, and employment conditions contributed to borrowers’ inability to make payments, especi

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [23]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [24]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
multi_query_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided context, the most common issues with loans appear to involve errors and misconduct by loan servicers, including:\n\n- Errors in loan balances and incorrect reporting to credit bureaus.\n- Mishandling of payments, including misapplied payments and inability to pay down principal.\n- Lack of transparency and inadequate communication about loan terms, interest, and account status.\n- Unauthorized access or breaches of borrower information, violating privacy laws like FERPA.\n- Improper collection practices and failure to adhere to federal regulations (e.g., failure to provide required notices, reporting defaults improperly).\n- Disputes over the enforceability of loan agreements due to administrative changes or material alterations.\n- Poor handling of forgiveness, discharge, or application processes.\n\nThe pattern suggests that administrative errors, misinformation, and lack of proper communication or documentation are among the most frequent and significant issue

In [26]:
multi_query_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, it appears that several complaints were not handled in a timely manner. For example:\n\n- One complaint mentioned it had been nearly 18 months with no resolution despite requests for review and corrections.\n- Other complaints indicated delays exceeding 30 days for investigations or responses, with some noting no response at all over extended periods.\n- Multiple complaints explicitly mention that the issues remain unresolved despite waiting for months or over a year, suggesting they were not managed promptly.\n\nTherefore, yes, several complaints did not get handled in a timely manner.'

In [27]:
multi_query_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans mainly because they faced financial hardships, were misled or lacked adequate information about repayment options, and encountered practices by servicers that pressured or steered them into unfavorable arrangements. Many borrowers did not qualify for forgiveness programs or were unaware of income-based repayment options, leading to prolonged periods of unmanageable debt. Additionally, some experienced issues like interest accumulation during forbearance, lack of proper notices, incorrect reporting to credit bureaus, or coercive servicing practices such as forbearance steering and incorrect loan handling, all of which contributed to their inability to repay their loans effectively.'

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

-
-

</div>

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [28]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = loan_complaint_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [29]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [30]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [31]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [32]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [33]:
parent_document_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'The most common issue with loans, based on the provided context, appears to be errors and misconduct related to federal student loan servicing. Specifically, problems such as incorrect information on reports, misapplied payments, wrongful denials of payment plans, discrepancies in loan balances and interest rates, and issues arising from loan transfers and agency dissolutions are frequently reported. Many complaints involve systemic breakdowns, misinformation, and unfair practices by loan servicers.'

In [34]:
parent_document_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Yes, based on the provided complaints, several did not get handled in a timely manner. Specifically, the complaints filed with MOHELA on 03/28/25 and 04/11/25 both were marked as "Timely response?": "No," indicating they were not responded to promptly. The complaint filed with Nelnet on 04/27/25, however, was responded to within the required timeframe ("Timely response?": "Yes"). \n\nTherefore, at least two complaints about unresolved issues did not receive timely responses.'

In [35]:
parent_document_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans primarily due to issues such as being unable to make payments during their studies or after graduation, often because of financial hardship or lack of proper information. Some specific reasons highlighted include:\n\n- Lack of clear communication or notification from loan servicers about payment obligations, such as when payments were supposed to start or changes in loan ownership.\n- Financial difficulties after graduation, including unemployment or severe financial hardship, which made it difficult to keep up with payments.\n- Misrepresentations or lack of transparency about the value of their education, the long-term consequences of taking out loans, or the stability of the educational institutions they attended.\n- Administrative errors or mishandling by loan servicing agencies, leading to reported delinquencies or incorrect credit reporting.\n- In some cases, borrowers were unaware that their payments had begun or were misled about their repa

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [36]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [37]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [38]:
ensemble_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

'Based on the provided complaints data, the most common issues with student loans appear to be related to:\n\n- Problems with loan servicing, including errors in loan balances, misapplied payments, and wrongful denials of payment plans.\n- Dealing with lenders or servicers due to miscommunication, lack of notification about account status, or inaccurate information.\n- Issues with how payments are being handled, including reversals, inability to apply payments correctly, and late reporting.\n- Incorrect or incomplete information reported to credit bureaus, impacting credit scores.\n- Challenges with understanding and managing loan terms, interest calculation, and repayment options.\n\nOverall, one of the most prevalent issues is **errors or mismanagement by loan servicers**, leading to incorrect balances, late payments reported to credit agencies, and difficulties in making or verifying payments. These servicing problems often cause financial hardship, damage to credit scores, and frus

In [39]:
ensemble_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided complaints, yes, some complaints were not handled in a timely manner. Specifically, there are at least two instances where the response was marked as "No" for being timely:\n\n1. Complaint ID: 12935889 (Page received 04/11/25, from MOHELA in CO) was marked as "Timely response? No."\n2. Complaint ID: 12744910 (Page received 03/31/25, from Maximus Federal Services, Inc. in MI) was marked as "Timely response? Yes," but the ongoing issues and delayed resolutions suggest the matter was not effectively handled promptly in reality.\n\nAdditionally, multiple complaints detail prolonged wait times, lack of responses, or failure to update the complainants within expected timeframes. Therefore, the answer is that yes, some complaints were not responded to or handled in a timely manner.'

In [40]:
ensemble_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for various reasons, including inadequate communication and notification from loan servicers, mismanagement of their loan information, lack of awareness about payment obligations, and difficulties caused by predatory practices such as forbearance steering and high interest accrual. Many borrowers were not properly informed about their repayment options, including income-driven plans or forgiveness programs, which led to unintentional delinquency or default. Additionally, some borrowers experienced issues like incorrect account information, transfer of loans without proper notification, or administrative errors that negatively impacted their credit and made repayment challenging.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [41]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [42]:
semantic_documents = semantic_chunker.split_documents(loan_complaint_data[:20])

Let's create a new vector store.

In [43]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="Loan_Complaint_Data_Semantic_Chunks"
)

We'll use naive retrieval for this example.

In [44]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [45]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [46]:
semantic_retrieval_chain.invoke({"question" : "What is the most common issue with loans?"})["response"].content

"Based on the provided context, the most common issue with loans appears to be related to problems with repayment and servicing. This includes issues such as:\n\n- Struggling to repay loans\n- Problems with loan forgiveness or discharge\n- Dealing with incorrect or bad information about loans\n- Trouble with how payments or payment plans are being handled\n- Incorrect account status or improper use of credit reports\n- Lack of transparency, communication issues, or disputes over loan details\n\nWhile the specific most frequent issue isn't explicitly stated, the majority of complaints revolve around difficulties in managing, understanding, or correctly servicing student loans. Therefore, the most common issue with loans, as evidenced by these complaints, is related to repayment problems and servicing errors."

In [47]:
semantic_retrieval_chain.invoke({"question" : "Did any complaints not get handled in a timely manner?"})["response"].content

'Based on the provided information, yes, some complaints did not get handled in a timely manner, specifically by Nelnet. The complaint from 05/04/25 regarding the transfer of a student loan account to Nelnet mentions that despite acknowledging receipt of the complaint via Certified Mail, Nelnet never responded to the correspondence. Additionally, the complaint from 05/04/25 about bad information about the loan also indicates that Nelnet responded with "Closed with explanation," suggesting the issue was not fully resolved promptly. \n\nWhile the responses to some complaints were marked as "timely" in the data, the fact that there are complaints indicating no response or unresolved issues suggests that not all complaints were handled in a timely or satisfactory manner.'

In [48]:
semantic_retrieval_chain.invoke({"question" : "Why did people fail to pay back their loans?"})["response"].content

'People failed to pay back their loans for various reasons, including issues related to miscommunication, administrative errors, or disputes over the legitimacy and status of their loans. Specific reasons highlighted in the complaints include:\n\n- Receiving incorrect or unclear information about loan status, such as being told about forbearance periods that were not documented in writing.\n- Difficulties in accessing account information and poor customer service, leading to frustration and unresolved issues.\n- Disputes over loan legitimacy or obligations, such as claims that loans are unverified or legally void due to administrative or legal issues.\n- Problems with repayment plans, including errors in payment processing or changes to payment amounts that borrowers can handle.\n- Allegations of improper reporting, breach of privacy, or illegal collection practices, which complicated the repayment process.\n- In some cases, borrowers found out their loans were in default due to admini

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

</div>

# 🤝 Breakout Room Part #2

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against each other. 
You can use the loans or bills dataset.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

</div>

##### HINTS:

- LangSmith provides detailed information about latency and cost.

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Analysis & Observations:

</div>